In [ ]:
import numpy as np
import tensorflow as tf
print(tf.__version__)


2.19.0


In [ ]:
with open('/content/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:500])  # preview text






                        THE ADVENTURES OF SHERLOCK HOLMES

                               Arthur Conan Doyle



                                Table of contents

               A Scandal in Bohemia
               The Red-Headed League
               A Case of Identity
               The Boscombe Valley Mystery
               The Five Orange Pips
               The Man with the Twisted Lip
               The Adventure of the Blue Carbuncle
               The Adventure of the Speckled Band
  


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

#Convert words into numbers (Tokenization)
tokenizer = Tokenizer()
#fit_on_texts([text]) -> Scans all words and builds vocabulary.
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
print("Total words:", total_words)


Total words: 8200


In [ ]:
input_sequences = []

for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        input_sequences.append(token_list[:i+1])

print("Total sequences:", len(input_sequences))


Total sequences: 96314


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

max_sequence_len = max(len(seq) for seq in input_sequences)

input_sequences = np.array(
    pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')
)


In [ ]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]


In [ ]:
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(X, y, epochs=50, batch_size=128)


Epoch 1/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.0516 - loss: 6.7631
Epoch 2/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 9s 12ms/step - accuracy: 0.0792 - loss: 5.9734
Epoch 3/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.1128 - loss: 5.6014
Epoch 4/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1319 - loss: 5.3366
Epoch 5/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1422 - loss: 5.1403
Epoch 6/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.1551 - loss: 4.9586
Epoch 7/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.1651 - loss: 4.7756
Epoch 8/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1771 - loss: 4.6221
Epoch 9/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1832 - loss: 4.4977
Epoch 10/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - accuracy: 0.1900 - loss: 4.3732
Epoch 11/50
753/753 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.1998 - loss: 4.2510
Epoch 12/50
753/753 ━━━━━━━━━━━━━━━━━━

In [ ]:
def predict_next_words(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences(
            [token_list], maxlen=max_sequence_len-1, padding='pre'
        )
        predicted = np.argmax(model.predict(token_list), axis=-1)

        for word, index in tokenizer.word_index.items():
            if index == predicted:
                seed_text += " " + word
                break
    return seed_text


print(predict_next_words("I will leave if they", 3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
I will leave if they are they in


In [ ]:
print(predict_next_words("I love ", 9))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
I love  and am loved by the nature of the amoy


In [ ]:
print(predict_next_words("i am ", 11))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
i am  afraid that i have not yet considerable experience of my own


In [ ]:
history = model.fit(
    X, y,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=1
)


Epoch 1/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.6007 - loss: 1.7939 - val_accuracy: 0.5500 - val_loss: 1.9818
Epoch 2/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.6107 - loss: 1.7363 - val_accuracy: 0.5233 - val_loss: 2.0842
Epoch 3/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6172 - loss: 1.7106 - val_accuracy: 0.5018 - val_loss: 2.1694
Epoch 4/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.6267 - loss: 1.6663 - val_accuracy: 0.4816 - val_loss: 2.2555
Epoch 5/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.6331 - loss: 1.6347 - val_accuracy: 0.4646 - val_loss: 2.3343
Epoch 6/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.6406 - loss: 1.6028 - val_accuracy: 0.4507 - val_loss: 2.4112
Epoch 7/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 14ms/step - accuracy: 0.6486 - loss: 1.5582 - val_accuracy: 0.4288 - val_loss: 2.4926
Epoch 8/30
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.6565 - loss: 1.5299 - val_acc

In [ ]:
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {train_acc:.4f}")
print(f"Final Validation Accuracy: {val_acc:.4f}")


Final Training Accuracy: 0.7591
Final Validation Accuracy: 0.2686


In [ ]:
import numpy as np

final_loss = history.history['loss'][-1]
perplexity = np.exp(final_loss)

print(f"Model Perplexity: {perplexity:.2f}")


Model Perplexity: 2.90
